In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
#from sklearn.model_selection import GridSearchCV !! muss auskommentiert sein für gpu grid search
from sklearn.metrics import classification_report, confusion_matrix
# für GPU ausführung:
import cudf
import cupy as cp
import gc
from cuml.neighbors import KNeighborsClassifier as cuKNN
from cuml.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from cuml.preprocessing import StandardScaler as cuStandardScaler
from sklearn.preprocessing import LabelEncoder
import itertools
from cuml import metrics

In [2]:
meta_train = pd.read_csv("/content/drive/MyDrive/bv-ss25-data/y_train.csv")
meta_val = pd.read_csv("/content/drive/MyDrive/bv-ss25-data/y_val.csv")
meta_test = pd.read_csv("/content/drive/MyDrive/bv-ss25-data/y_test.csv")

embeddings_train =  np.load("/content/drive/MyDrive/bv-ss25-data/emb_train.npy")  # shape (N, d)
embeddings_val =  np.load("/content/drive/MyDrive/bv-ss25-data/emb_val.npy")  # shape (N, d)
embeddings_test =  np.load("/content/drive/MyDrive/bv-ss25-data/emb_test.npy")  # shape (N, d)

In [3]:
for i in range(embeddings_train.shape[1]):
  meta_train[f"feat_{i}"] = embeddings_train[:, i]


for i in range(embeddings_val.shape[1]):
  meta_val[f"feat_{i}"] = embeddings_val[:, i]


for i in range(embeddings_test.shape[1]):
  meta_test[f"feat_{i}"] = embeddings_test[:, i]

/tmp/ipython-input-92549215.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  meta_train[f"feat_{i}"] = embeddings_train[:, i]
/tmp/ipython-input-92549215.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  meta_train[f"feat_{i}"] = embeddings_train[:, i]
/tmp/ipython-input-92549215.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented fr

In [4]:
# Embeddings direkt verwenden (und gleich in float32 casten, um den Speicher zu halbieren)

# Trainingsdaten
X_train = embeddings_train.astype(np.float32)
y_train = meta_train["country"].values

# Validierungsdaten
X_val = embeddings_val.astype(np.float32)
y_val = meta_val["country"].values

# Testdaten
X_test = embeddings_test.astype(np.float32)
y_test = meta_test["country"].values


# Freigeben ungenutzer objekte -> mehr ram
del meta_train, meta_val, meta_test
del embeddings_train, embeddings_val, embeddings_test


In [5]:

# 1. Labels encodieren
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)   # NumPy-Array int64
y_val_enc   = le.transform(y_val)
y_test_enc  = le.transform(y_test)

# 2. In int32 casten (passt besser zu GPU)
y_train_enc = y_train_enc.astype(np.int32)
y_val_enc   = y_val_enc.astype(np.int32)
y_test_enc  = y_test_enc.astype(np.int32)

In [6]:
# CPU
scaler = StandardScaler().fit(X_train)
X_train_s = scaler.fit_transform(X_train)
X_val_s = scaler.transform(X_val)
X_test_s = scaler.transform(X_test)

# Auf GPU geht nicht wegen RAM, deshalb hier mit CPU und danach auf GPU
# Freigeben ungenutzer objekte -> mehr ram
del X_train, X_val, X_test

# aus float32-NumPy → float32-CuPy
X_train_gpu = cp.asarray(X_train_s)
y_train_gpu = cp.asarray(y_train_enc)

X_val_gpu   = cp.asarray(X_val_s)
y_val_gpu   = cp.asarray(y_val_enc)

X_test_gpu  = cp.asarray(X_test_s)
y_test_gpu  = cp.asarray(y_test_enc)


In [7]:
# 1. Rapids/cuML in Colab installieren
# -------------------------------------
# (Muss nur einmal pro Session laufen)
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/install_rapids.sh stable

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 597, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 597 (delta 128), reused 82 (delta 82), pack-reused 434 (from 3)
Receiving objects: 100% (597/597), 196.59 KiB | 10.35 MiB/s, done.
Resolving deltas: 100% (302/302), done.
bash: rapidsai-csp-utils/colab/install_rapids.sh: No such file or directory


In [ ]:
# 1) Param-Grid
param_grid = {
    "n_neighbors": [3,5,7,9,11],
    "weights":     ["uniform"],
    "metric":      ["euclidean","manhattan"]
}

best_score = -1.0
best_params = None

# 2) schleife über alle Kombinationen
for k, w, m in itertools.product(param_grid["n_neighbors"],
                                 param_grid["weights"],
                                 param_grid["metric"]):

    # Modell initialisieren & trainieren
    clf = cuKNN(n_neighbors=k, weights=w, metric=m)
    clf.fit(X_train_gpu, y_train_gpu)

    # auf Val-Set vorhersagen & Accuracy berechnen
    y_val_pred_gpu = clf.predict(X_val_gpu)
    acc = metrics.accuracy_score(y_val_gpu, y_val_pred_gpu)

    # bestes Modell merken
    if acc > best_score:
        best_score = acc
        best_params = {"n_neighbors": k, "weights": w, "metric": m}

    # Speicher freigeben für nächsten Durchlauf
    del clf, y_val_pred_gpu
    gc.collect()
    cp._default_memory_pool.free_all_blocks()

print(f" Best Validation Accuracy: {best_score:.4f}")
print(" Beste Parameter:", best_params)

# 3) Finales Modell mit den besten Parametern trainieren
best_knn = cuKNN(**best_params)
best_knn.fit(X_train_gpu, y_train_gpu)

# 4) val prediction holen
val_pred_gpu = best_knn.predict(X_val_gpu)

# 5) In NumPy und Encodings zurückholen
val_pred_enc = val_pred_gpu.get().astype(int)
val_pred_labels = le.inverse_transform(val_pred_enc)

# 6) Metriken ausgeben
print("\n=== Validation Classification Report ===")
print(classification_report(y_val, val_pred_labels, zero_division=0))
print("=== Validation Confusion Matrix ===")
print(confusion_matrix(y_val, val_pred_labels))


# 7) Aufräumen
del val_pred_gpu, val_pred_enc
gc.collect()
cp._default_memory_pool.free_all_blocks()


 Best Validation Accuracy: 0.6975
 Beste Parameter: {'n_neighbors': 9, 'weights': 'uniform', 'metric': 'euclidean'}

=== Validation Classification Report ===
              precision    recall  f1-score   support

          AT       0.51      0.57      0.54      4999
          BE       0.60      0.60      0.60      4996
          BG       0.80      0.88      0.84      4767
          CH       0.70      0.63      0.67      5000
          CZ       0.65      0.56      0.60      4998
          DE       0.45      0.54      0.49     10000
          DK       0.56      0.60      0.58      5000
          EE       0.75      0.83      0.79      4992
          ES       0.67      0.79      0.72      9898
          FI       0.74      0.84      0.79      4999
          FR       0.61      0.55      0.58     10000
          GB       0.72      0.85      0.78     10000
          GR       0.81      0.81      0.81      4999
          HR       0.72      0.57      0.64      4995
          HU       0.64      0.

NameError: name 'val_pred' is not defined

In [ ]:
train_fracs = [0.1, 0.3, 0.5, 0.7, 1.0]
train_scores, val_scores = [], []

for frac in train_fracs:
    # Subset-Größe
    n = int(frac * frac * X_train_gpu.shape[0])
    X_sub, y_sub = X_train_gpu[:n], y_train_gpu[:n]

    # Neues KNN mit den besten Parametern
    clf = cuKNN(**best_params)
    clf.fit(X_sub, y_sub)

    # Accuracy auf Subset (Train) & Val
    # Training-Accuracy
    y_tr_pred = clf.predict(X_sub)
    tr_acc = metrics.accuracy_score(y_sub, y_tr_pred)

    # Validation-Accuracy
    y_va_pred = clf.predict(X_val_gpu)
    va_acc = metrics.accuracy_score(y_val_gpu, y_va_pred)

    train_scores.append(tr_acc)
    val_scores.append(va_acc)

    # Speicher freigeben
    del clf, y_tr_pred, y_va_pred
    gc.collect()
    cp._default_memory_pool.free_all_blocks()



In [ ]:
# 2) Plot der Lernkurve
plt.figure(figsize=(8, 5))
plt.plot(train_fracs, train_scores, marker='o', color="#1e34dc", label="Train")
plt.plot(train_fracs, val_scores,   marker='o', color="#fdd210" , label="Validation")
plt.xlabel("Anteil des Trainingssets")
plt.ylabel("Accuracy")
plt.title("Lernkurve: KNN (GPU mit cuML)")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 3) Finale Evaluation auf dem Test-Set
# Modell auf vollem Trainingsset fitten
final_knn = cuKNN(**best_params)
final_knn.fit(X_train_gpu, y_train_gpu)

# Vorhersagen auf Test-Set
y_test_pred_gpu = final_knn.predict(X_test_gpu)

# In NumPy-Array umwandeln, encoden und zurück in Strings
y_test_pred_enc    = y_test_pred_gpu.get().astype(int)
y_test_pred_labels = le.inverse_transform(y_test_pred_enc)

# 4) Classification Report & Confusion Matrix
print("=== Test Classification Report ===")
print(classification_report(y_test, y_test_pred_labels, zero_division=0))
print("=== Test Confusion Matrix ===")
print(confusion_matrix(y_test, y_test_pred_labels))